CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from datetime import datetime
from tensorflow.keras.utils import plot_model
from IPython.display import Image
from tensorflow.keras.models import load_model


In [ ]:
%cd '/content/gdrive/My Drive/.../'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Create Network
classifier = Sequential()

In [ ]:
#Step 1: Convolution
classifier.add(Conv2D(filters = 32,kernel_size = 3,input_shape=(64,64,3),activation = 'relu'))

In [ ]:
#Step 2: Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
#Adding second covolution layer
classifier.add(Conv2D(filters = 32,kernel_size = 3,activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))


In [ ]:
#Step 3: Flattenning
classifier.add(Flatten())


In [ ]:
#Step 4: Full Connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))/nếu 3 class thì đổi thành softmax
classifier.summary()


In [ ]:
#compiling CNN
classifier.compile(optimizer='adam',
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])


In [ ]:
#Read and Preprocessing Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set=train_datagen.flow_from_directory('dataset/training_set',
                                             target_size = (64,64),
                                             batch_size = 32,
                                             class_mode = 'binary') #if 3-> ‘categorical’
test_set =test_datagen.flow_from_directory('dataset/test_set',
                                              target_size = (64,64),
                                              batch_size = 32,
                                              class_mode = 'binary') #if 3-> ‘categorical’


In [ ]:
#Fit CNN to Images
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [EarlyStopping(monitor = 'val_loss',
                           patience = 20),
             ModelCheckpoint('model_checkpoit_20082021.h5',
                             save_best_only = True)
             ]
from datetime import datetime
d1 = datetime.datetime.now()
history = classifier.fit(
    training_set,
    epochs = 100,
    validation_data = test_set,
    callbacks = callbacks
)
d2 = datetime.datetime.now()


In [ ]:
#Plot
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:,['loss','val_loss']].plot()
history_frame.loc[:,['accuracy','val_accuracy']].plot()
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(classifier,to_file='CNN_model_20082021.png',show_shape=True)
Imag(filename = 'CNN_model_20082021.png')


In [ ]:
#Save model
from tensorflow.keras.models import load_model
classifier.save('my_model_CNN_20082021.h5')


In [ ]:
#	Prediction
from tensorflow.keras.models import load_model
classifier = load_model('my_model_CNN_20082021.h5')
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_image('dataset/single_prediction/cat_or_dog_2.jpg',
                              target_size = (64,64))
test_image = image.img_to_array(test_image)
print(test_image.shape)
test_image = test_image/255
test_image = np.expand_dims(test_image,axis = 0)
print(test_image.shape)
result = classifier.predict(test_image)
print(result)


In [ ]:
# 0: cat, 1: dog, sigmoid threshold = 0.5
if result[0][0]>0.5:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)


In [ ]:
#show image
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('dataset/single_prediction/cat_or_dog_2.jpg')
imgplot = plt.imshow(img)
plt.show()